In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='tuwen'

In [4]:
# 载入模型
model_hour = '2020043019'
valid_hour = '2020043022'
# valid_hour = '2020030812'
model_root = f'/search/odin/publicData/CloudS/libowei/rank_online/data/{mark}_hour_sgsapp_v1/models/8'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_root = f'/search/odin/publicData/CloudS/libowei/rank/{product}/data/{mark}_hour_{product}_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
tfrecord_dir2 = f'{tfrecord_root}/{model_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020043019
record: /search/odin/publicData/CloudS/libowei/rank/sgsapp/data/tuwen_hour_sgsapp_v1/tfrecords/2020043022


2020-05-25 17:36:10 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-05-25 17:36:10 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-05-25 17:36:10 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-05-25 17:36:10 0:00:00 model: [8] model_dir: [/search/odin/libowei/rank/tuwen/model/8]
2020-05-25 17:36:10 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020043019/model.ckpt-2020043019-1.00-8533


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

2157706

In [7]:
batch_size = 512
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [8]:
def gen_feats(batches, num_steps, ofile):
  with open(ofile, 'w') as out:
    print('mid,impression_time,docid,position,dur,click_feat,dur_feat', file=out)
    for x, _ in tqdm(batches, total=num_steps):
      mids = gezi.decode(x['mid'].numpy())
      docids = gezi.decode(x['docid'].numpy())
      positions = x['position'].numpy()
      durs = gezi.squeeze(x['duration'].numpy())
      impression_times = gezi.squeeze(x['impression_time'].numpy())
      model(x)
      click_feats = model.deep.click_feat.numpy()
      dur_feats = model.deep.dur_feat.numpy()
      for mid, impression_time, docid, position, dur, click_feat, dur_feat in zip(mids, impression_times, docids, positions, durs, click_feats, dur_feats):
        print(mid, impression_time, docid, position, dur, ' '.join(map(str, click_feat)), ' '.join(map(str, dur_feat)), sep=',', file=out)

In [9]:
gen_feats(batches, num_steps, f'/home/gezi/tmp/rerank/feats/valid/{valid_hour}.txt')

100%|██████████| 4215/4215 [09:58<00:00,  7.05it/s]


In [10]:
files = gezi.list_files(tfrecord_dir2)
total = melt.get_num_records(files) 
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)
gen_feats(batches, num_steps, f'/home/gezi/tmp/rerank/feats/train/{model_hour}.txt')

100%|██████████| 3119/3119 [07:13<00:00,  7.19it/s]    


In [11]:
train_df = pd.read_csv(f'/home/gezi/tmp/rerank/feats/train/{model_hour}.txt')

In [12]:
len(train_df)

1596689

In [13]:
train_df['impression'] = train_df['mid'] + '\t' + train_df['impression_time'].astype(str)

In [14]:
train_df = train_df.sort_values(['impression', 'position'])

In [15]:
mid = train_df.mid.sample(1).values[0]

In [16]:
train_df[train_df.mid==mid]

,mid,impression_time,docid,position,dur,click_feat,dur_feat,impression
68047,de7f867950045807086,1588245135,open_doc_prod27540656,0,218,-1.6365812 -1.4301285 -0.7069505 -2.343216 -0....,-2.3405602 -5.664321 -6.350601 -5.566018 -0.92...,de7f867950045807086\t1588245135
1285422,de7f867950045807086,1588245135,open_doc_prod27724876,1,0,-1.7943183 -1.2030618 -0.4425044 -0.7907157 -1...,-1.8383176 -10.473582 -12.092896 -10.656795 -1...,de7f867950045807086\t1588245135
1079178,de7f867950045807086,1588245135,20429c0l2PZEuW,2,0,-3.7807562 -1.2230141 -1.1156911 543.734 -2.45...,-3.2276545 -10.424248 -13.18947 -10.808464 -1....,de7f867950045807086\t1588245135
1186290,de7f867950045807086,1588245135,20429l0x4dUbGA,3,33,-1.5936571 -1.3178418 -0.8805831 -0.8176816 -1...,-2.7972407 -4.9881244 -5.6084886 -4.9197025 -1...,de7f867950045807086\t1588245135
52637,de7f867950045807086,1588245135,20428a0l49c3AH,4,21,-0.60245734 -0.8149589 -0.1497002 62.737595 -0...,-2.2310836 -5.3985367 -5.7761893 -5.1697297 -0...,de7f867950045807086\t1588245135
199366,de7f867950045807086,1588245135,2042780l465maH,5,0,-1.1701341 -0.95317656 -0.6209997 158.20262 -0...,-3.186306 -3.6369493 -4.0698237 -3.4673424 -0....,de7f867950045807086\t1588245135
109018,de7f867950045807086,1588245135,2043080x2CGyqJ,7,0,-1.4654723 -1.2087358 -0.8051475 128.18808 -1....,-3.4984505 -3.4486258 -3.8840063 -3.2082841 -1...,de7f867950045807086\t1588245135
341956,de7f867950045807086,1588245135,20200428A096HT00,9,0,-1.3844786 -1.2255849 -0.95889825 110.28902 -1...,-3.776055 -2.0702202 -2.2086313 -1.8710661 -1....,de7f867950045807086\t1588245135
206906,de7f867950045807086,1588245135,20427m0l0PuLl2,11,0,-1.207714 -1.08409 -0.63307434 140.94366 -0.83...,-3.405423 -3.0638952 -3.4284832 -2.8791275 -0....,de7f867950045807086\t1588245135
496865,de7f867950045807086,1588245135,open_doc_prod27762664,13,0,-1.9303249 -0.97165036 -0.66612136 333.16928 -...,-3.4164102 -4.6314507 -5.8282714 -4.6408963 -1...,de7f867950045807086\t1588245135


In [17]:
impressions = train_df.impression.values

In [18]:
def gen_records(df, ofile):
  with melt.tfrecords.Writer(ofile) as writer:
    pre_impression = None
    feature = {}
    click_feats = []
    dur_feats = []
    positions = []
    durs = []
    mids = []
    num_records = 0
    for i in tqdm(range(len(df))):
      row = df.iloc[i]
      impression = row['impression']
      if impression != pre_impression:
        if durs:
          feature['durs'] = melt.gen_feature(durs, np.int64)
          feature['positions'] = melt.gen_feature(positions, np.int64)
          feature['impression'] = melt.gen_feature(row['impression'], df['impression'].dtype)
          feature['click_feats'] = melt.gen_feature(click_feats, np.float32)
          feature['dur_feats'] = melt.gen_feature(dur_feats, np.float32)
          feature['mids'] = melt.gen_feature(mids, df['mid'].dtype)
          record = tf.train.Example(features=tf.train.Features(feature=feature))
          writer.write(record)
          num_records += 1
        feature = {}
        click_feats = []
        dur_feats = []
        positions = []
        durs = []
        mids = []
        pre_impression = impression
      else:  
        click_feats += list(map(float, row['click_feat'].split()))
        dur_feats += list(map(float, row['dur_feat'].split()))
        positions += [row['position'] + 1]
        durs += [row['dur']]
        mids += [row['mid']]
        
    if durs:
      feature['durs'] = melt.gen_feature(durs, np.int64)
      feature['positions'] = melt.gen_feature(positions, np.int64)
      feature['impression'] = melt.gen_feature(row['impression'], df['impression'].dtype)
      feature['click_feats'] = melt.gen_feature(click_feats, np.float32)
      feature['dur_feats'] = melt.gen_feature(dur_feats, np.float32)
      feature['mids'] = melt.gen_feature(mids, df['mid'].dtype)
      record = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(record)
      num_records += 1
      
    with open(os.path.join(os.path.dirname(ofile), 'num_records.txt'), 'w') as out:
      print(num_records, file=out)

In [19]:
gen_records(train_df, f'/home/gezi/tmp/rerank/tfrecords/train/{model_hour}/0.record')

100%|██████████| 1596689/1596689 [11:36<00:00, 2293.79it/s]


In [20]:
valid_df = pd.read_csv(f'/home/gezi/tmp/rerank/feats/valid/{valid_hour}.txt')
valid_df['impression'] = valid_df['mid'] + '\t' + valid_df['impression_time'].astype(str)
valid_df = valid_df.sort_values(['impression', 'position'])

In [21]:
gen_records(valid_df, f'/home/gezi/tmp/rerank/tfrecords/valid/{valid_hour}/0.record')

100%|██████████| 2157706/2157706 [15:38<00:00, 2298.84it/s]
